In [1]:
!pip install keras-gcn

In [2]:
import keras
from keras_gcn import GraphConv
import networkx as nx
import numpy as np
from sklearn import preprocessing

Using TensorFlow backend.


In [3]:
zkc = nx.karate_club_graph()
order = sorted(list(zkc.nodes))

In [4]:
zkc.nodes

NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33))

In [5]:
DATA_DIM = 1
data_layer = keras.layers.Input(shape=(None, DATA_DIM), name="data_layer")
edge_layer = keras.layers.Input(shape=(None, None), name="edge_layer")
conv_layer = GraphConv(units=2, step_num=1, name="graph_conv_layer")([data_layer, edge_layer])
output_layer = keras.layers.Dense(1, name="sigmoid_layer")(conv_layer)

W0724 19:40:37.679340 140476122761024 deprecation_wrapper.py:119] From /home/aaruran/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0724 19:40:37.925109 140476122761024 deprecation_wrapper.py:119] From /home/aaruran/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [6]:
model = keras.models.Model(inputs=[data_layer, edge_layer], outputs=output_layer)
model.compile(optimizer="adam",
             loss='mae',
             metrics=['mae'])
model.summary()

W0724 19:40:42.057561 140476122761024 deprecation_wrapper.py:119] From /home/aaruran/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0724 19:40:42.059289 140476122761024 deprecation_wrapper.py:119] From /home/aaruran/miniconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data_layer (InputLayer)         (None, None, 1)      0                                            
__________________________________________________________________________________________________
edge_layer (InputLayer)         (None, None, None)   0                                            
__________________________________________________________________________________________________
graph_conv_layer (GraphConv)    (None, None, 2)      4           data_layer[0][0]                 
                                                                 edge_layer[0][0]                 
__________________________________________________________________________________________________
sigmoid_layer (Dense)           (None, None, 1)      3           graph_conv_layer[0][0]           
Total para

In [7]:
input_data = np.array([list(zkc.nodes)])
input_data = input_data[:, :, np.newaxis]
input_data.shape

(1, 34, 1)

In [10]:
zkc.node[0]['club']

'Mr. Hi'

In [11]:
input_edge = nx.to_numpy_matrix(zkc, nodelist=order)
input_edge = np.array([input_edge]).T
input_edge.shape
# input_edge

(34, 34, 1)

In [12]:
le = preprocessing.LabelEncoder()
labels = le.fit_transform([zkc.node[n]['club'] for n in zkc.node])
labels = labels[np.newaxis, :, np.newaxis]
labels.shape

(1, 34, 1)

In [ ]:
x_train = labels[0::2]

In [13]:
model.predict([input_data, input_edge])

W0724 19:44:00.741288 140476122761024 deprecation_wrapper.py:119] From /home/aaruran/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:2741: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0724 19:44:00.794501 140476122761024 deprecation_wrapper.py:119] From /home/aaruran/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



array([[[-1.7711055e+09],
        [-1.7711053e+02],
        [-1.7711053e+02],
        [-1.7711053e+02],
        [-1.7711053e+02],
        [-1.7711053e+02],
        [-1.7711053e+02],
        [-1.7711053e+02],
        [-1.7711053e+02],
        [-1.7711055e+09],
        [-1.7711053e+02],
        [-1.7711053e+02],
        [-1.7711053e+02],
        [-1.7711053e+02],
        [-1.7711055e+09],
        [-1.7711055e+09],
        [-1.7711055e+09],
        [-1.7711053e+02],
        [-1.7711055e+09],
        [-1.7711053e+02],
        [-1.7711055e+09],
        [-1.7711053e+02],
        [-1.7711055e+09],
        [-1.7711055e+09],
        [-1.7711055e+09],
        [-1.7711055e+09],
        [-1.7711055e+09],
        [-1.7711055e+09],
        [-1.7711055e+09],
        [-1.7711055e+09],
        [-1.7711055e+09],
        [-1.7711053e+02],
        [-1.7711055e+09],
        [-1.7711055e+09]]], dtype=float32)

In [16]:
model.fit(x=[input_data, input_edge], y=labels, epochs=10, batch_size=64)

ValueError: All input arrays (x) should have the same number of samples. Got array shapes: [(1, 34, 1), (34, 34, 1)]

In [ ]:
input_data